In [9]:
# Contextual_Risk_Reasoning

In [10]:
# Goal:
# Generate an evidence-grounded explanation of clinical risk
# using only retrieved guideline excerpts.

# No recommendations.
# No external knowledge.

In [11]:
# Load environment variables
from dotenv import load_dotenv
load_dotenv()


True

In [12]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.3-70b-versatile")

In [13]:
# Output Schema

In [14]:
from pydantic import BaseModel
from typing import List, Literal

class EvidenceCitation(BaseModel):
    source: str
    page: int

class RiskAnalysis(BaseModel):
    summary: str
    mechanism: str
    evidence: List[EvidenceCitation]
    risk_level: Literal["low", "moderate", "high"]


In [15]:
# Simulated Inputs

In [16]:
patient_context = {
    "age": 65,
    "conditions": ["Chronic Kidney Disease Stage 3"],
    "risk_factors": ["renal impairment"]
}

medication_context = {
    "drug_name": "Ibuprofen",
    "total_daily_dose_mg": 2400,
    "duration_days": 5
}


In [17]:
retrieved_guidelines = [
    {
        "source": "NSAIDs_and_Kidney_Health.pdf",
        "page": 3,
        "content": "Prolonged or high-dose use of NSAIDs can lead to acute kidney injury, particularly in patients with pre-existing chronic kidney disease due to reduced renal blood flow."
    },
    {
        "source": "NSAIDs_and_Kidney_Health.pdf",
        "page": 0,
        "content": "Patients with chronic kidney disease are more susceptible to NSAID-induced nephrotoxicity, especially at higher doses or with prolonged use."
    }
]



In [18]:
guideline_text = "\n\n".join(
    f"Source: {g['source']}, Page: {g['page']}\n{g['content']}"
    for g in retrieved_guidelines
)


In [19]:
# 4️⃣ Build the Reasoning Chain

In [20]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser


In [21]:
parser = PydanticOutputParser(pydantic_object=RiskAnalysis)


In [22]:
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "You are a clinical risk reasoning system."),
    ("human", 
     """TASK:
Explain the clinical risk of the proposed medication in the given patient context.

IMPORTANT RULES:
- Use ONLY the guideline excerpts provided below.
- Do NOT introduce medical knowledge not present in the excerpts.
- Do NOT give recommendations or clinical instructions.
- If the excerpts do not provide sufficient evidence, state that clearly.
- Keep the tone clinical and non-alarmist.
- Output MUST strictly match the JSON schema.
- Do NOT include any text outside the JSON.

PATIENT CONTEXT:
{patient_context}

PROPOSED MEDICATION:
{medication_context}

GUIDELINE EXCERPTS:
{guideline_text}

JSON SCHEMA:
{format_instructions}
""")
])


In [23]:
risk_chain = prompt | llm | parser


In [24]:
risk_analysis = risk_chain.invoke({
    "patient_context": patient_context,
    "medication_context": medication_context,
    "guideline_text": guideline_text,
    "format_instructions": parser.get_format_instructions()
})


In [25]:
risk_analysis


RiskAnalysis(summary="The proposed medication, Ibuprofen, may pose a risk to the patient's kidney health due to their pre-existing chronic kidney disease and renal impairment.", mechanism='Reduced renal blood flow and NSAID-induced nephrotoxicity', evidence=[EvidenceCitation(source='NSAIDs_and_Kidney_Health.pdf', page=3), EvidenceCitation(source='NSAIDs_and_Kidney_Health.pdf', page=0)], risk_level='high')

In [30]:
# from IPython.display import JSON
# JSON(risk_analysis.model_dump_json())

In [39]:
from IPython.display import JSON, display

display(JSON(risk_analysis.model_dump_json()))

<IPython.core.display.JSON object>

In [31]:
print(risk_analysis.model_dump_json(indent=2))


{
  "summary": "The proposed medication, Ibuprofen, may pose a risk to the patient's kidney health due to their pre-existing chronic kidney disease and renal impairment.",
  "mechanism": "Reduced renal blood flow and NSAID-induced nephrotoxicity",
  "evidence": [
    {
      "source": "NSAIDs_and_Kidney_Health.pdf",
      "page": 3
    },
    {
      "source": "NSAIDs_and_Kidney_Health.pdf",
      "page": 0
    }
  ],
  "risk_level": "high"
}
